<a href="https://colab.research.google.com/github/GisstHubana/FHIR_Reporting_Tool/blob/GisstHubana-patch-html/Code/fhir_reporting_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorbereitung
## CleanUp - Eventuell gespeicherte Variablen löschen vor Ausführung des Colab books für eine bessere Reproduzierbarkeit

In [1]:
%reset -f

## Installieren notwendiger Pakete

In [2]:
!pip install pyngrok
!pip install flask-ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=abc718f977105ee9778f03f9666d552065b7f1accb502e7cb411f6910dc22eda
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


## gnrok mit Authorizierungstoken verknüpfen

In [3]:
!ngrok authtoken 2Wlt2vGzytTpJ6zABKGKKwfiTof_7y4sewzGkNGy7zW6GB5oz

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Importieren der notwendigen Libraries

In [4]:
from flask import Flask, request
from flask.templating import render_template
from flask_ngrok import run_with_ngrok
import secrets
import sqlite3

Erstellung einer virtuellen Kopie des GitHub Repositorium FHIR_Reporting_Tool in Colab

In [5]:
%cd /content # Wechsel in den Ordner /content - für den Fall, dass in einem Unterordner gearbeitet wurde

!git clone https://github.com/GisstHubana/FHIR_Reporting_Tool.git #virtuelle Kopie des Repositoriums - Bei Neustart zuvor Runtime/disconnect and delete runtime ausführen
!ls # Auflistung aller Ordnerinhalte

[Errno 2] No such file or directory: '/content # Wechsel in den Ordner /content - für den Fall, dass in einem Unterordner gearbeitet wurde'
/content
Cloning into 'FHIR_Reporting_Tool'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 298 (delta 130), reused 23 (delta 23), pack-reused 98
Receiving objects: 100% (298/298), 66.57 KiB | 3.33 MiB/s, done.
Resolving deltas: 100% (162/162), done.
FHIR_Reporting_Tool  sample_data


### Database in Flask

#### Use database to match login credentials

In [6]:
import sqlite3
db_folder = "/content/FHIR_Reporting_Tool/"
conn = sqlite3.connect(db_folder+"mydatabase.db") # falls Datenbank noch nicht existiert wird sie mit diesem Aufruf erzeugt
cursor = conn.cursor()

#query
create_table_query = """
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  username TEXT NOT NULL,
  password TEXT NOT NULL

)

"""
cursor.execute(create_table_query)

insert_query = """
INSERT INTO users(username, password)
SELECT ?, ?
WHERE NOT EXISTS (
  SELECT 1 FROM users WHERE username = ?

)
"""
user_data = ("USER3", "1234", "USER3")
cursor.execute(insert_query, user_data)
conn.commit()

select_query = "SELECT * FROM users"
cursor.execute(select_query)
rows = cursor.fetchall()
for row in rows:
  print(row)


(1, 'USER3', '1234')


In [ ]:
template_folder = "/content/FHIR_Reporting_Tool/templates"
static_folder = "/content/FHIR_Reporting_Tool/static"
db_folder = "/content/FHIR_Reporting_Tool/"
app = Flask(__name__, template_folder=template_folder, static_folder=static_folder) # Flask-App, Benennung template-Ordner
# app.debug = True # Debug-Modus aktivieren - webseite lädt nicht wenn eingeschaltet
app.secret_key = secrets.token_hex(32)
run_with_ngrok(app) # run_with_ngrok Funktion hinzufügen, um App mit ngrok auszuführen

@app.route('/', methods=["GET", "POST"])
def login():

  if request.method == "POST":
    username = request.form.get("username")
    password = request.form.get("password")

    conn = sqlite3.connect(db_folder+"mydatabase.db") # falls Datenbank noch nicht existiert wird sie mit diesem Aufruf erzeugt
    cursor = conn.cursor()
    # query
    query = """
    SELECT * FROM users WHERE username = ? AND password = ?
    """
    cursor.execute(query, (username, password))
    result = cursor.fetchone()

    if result:
     return render_template("page2.html", name=username)
    else:
      message = "Wrong username or password. Please try again!"
      return render_template("login.html", message=message)

  else:
    return render_template("login.html")

@app.route('/home') # Erstellung einer Routendefinition home
def index():
    return render_template("index.html")

@app.route('/page2')
def page2():
    username = input("Enter your name:")
    return render_template("page2.html", name=username)

@app.route('/page3')
def page3():
    return render_template("page3.html")

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://742d-34-32-198-57.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:45:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:45:41] "GET /static/styles/myfile.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:45:42] "GET /static/js/myjs.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:45:42] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:45:47] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:45:47] "GET /static/styles/myfile.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:45:47] "GET /static/js/myjs.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:45:57] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:45:57] "GET /static/js/myjs.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:45:57] "GET /static/styles/myfile.css HTTP/1.1" 304 -


## Abschluss - Runtime löschen und abschließend clear all outputs

In [ ]:
# Bei Neustart zuvor Runtime/disconnect and delete runtime ausführen
from google.colab import runtime
runtime.unassign() # delete runtime